---
title: "Reverse dictionary"
format: 
  html:
    code-fold: true
jupyter: python3
project:
  type: website
  output-dir: docs
---

## Diccionario español inverso parte 1: Embeddings

La idea es poner una frase que vagamente se parezca a la palabra que te quieres acordar, y ver qué sugerencias te da el modelo. Me pasa relativamente seguido y googlear a la mala generalmente no da buenos resultados. Un diccionario inverso (reverso?) es justamente uno donde a partir de una definición se obtiene la palabra que más se acerca a esa definición. Ahora con el boom de los LLM (large language models) supuse que sería más fácil que cuando quería algo asi hace tiempo. Como suele pasar eso si, los recursos en español están rezagados respecto al inglés y por lo mismo construir algo así no es tan directo. 

En esta primera parte voy armar la versión más simple es con embeddings. A quién le interese la explicación y la dificultad ridícula de armar esta demo, y por qué les tira error con algunas cosas puede leerlo mas abajo. Si no, puede jugar un poco con la app a ver que le sale, y leer un poco sobre los ejemplos que puse, con sus variados resultados. Al más interesado puede ver el notebook con el código [acá](https://github.com/rrrventures/reverse_dictionary_spa/blob/main/reverse%20dictionary.ipynb)


<script
	type="module"
	src="https://gradio.s3-us-west-2.amazonaws.com/3.41.2/gradio.js"
></script>

<gradio-app src="https://ilde-reverse-dictionary.hf.space"></gradio-app>

## Ejemplos

Igual para que tengan sentido los ejemplos, una explicación corta de los embeddings: Los embeddings de palabras son una lista de números que capturan el significado de las palabras de forma que los computadores pueden entender y manejar. Como son números, permite realizar operaciones matemáticas para determinar similitud (ej: distancia) y por tanto ver qué palabras se parecen más, o menos, a otras.

##### Ejemplo 0: "actor mujer"

Variación del ejemplo clásico para los embeddings (rey + mujer). Como es de esperarse, te sugiere actriz.  

##### Ejemplo 1: 'tardanza o lentitud con que se hace algo'

En este caso funciona relativamente bien: retraso, celeridad, otras formas de tardanza

##### Ejemplo 2: 'miedo a las alturas'

Verdaderamente esperaba ver vértigo, pero resulta que en verdad la palabra correcta es acrofobia. Si bien no es la primera sugerencia, aparece en la lista asi queeeeee bien igual.

##### Ejemplo 3: 'vehículo que anda bajo el agua'

La idea era que saliera submarino. Chistoso que la sugerencia principal sea desbarrancada. Claramente el vehículo no debería estar bajo el agua. Falló pero igual le doy puntos por el valor cómico

##### Ejemplo 4: 'grupo de lobos que andan juntos'

Aquí falló con escándalo. Esperaba que saliera manada o algo por el estilo pero salió albertosaurio que ni sabía si era un dinosaurio real o no. Tiene sentido en todo caso, por cómo funcionan los embeddings. Las palabras tienen una sola lista de números asociadas a cada una (un significado) siendo que en realidad pueden tener múltiples significados de acuerdo al contexto. Grupo y andan puede ser cualquier cosa, y se encuentran 'cercanas' a otros conceptos más literales. De hecho, si uno pone 'rebaño de lobos' efectivamente la primera sugerencia es manada (primero probé cardumen de lobos, no funcionó jeje). 

## ELI5 Embeddings

Imagina que tienes una cesta de frutas como manzanas, plátanos y naranjas, y quieres describir cada una sin usar sus nombres. Podrías decir que una manzana es roja, crujiente y dulce. Un plátano podría ser amarillo, suave y también dulce. Una naranja podría ser naranja, jugosa y un poco ácida.

En este ejemplo, las palabras rojo, crujiente, dulce, amarillo, suave, ácido y jugoso pueden servir como características para describir cada fruta. Cada una de estas características puede considerarse una dimensión, como las dimensiones X, Y y Z en el espacio físico que te rodea.

Los embeddings de palabras en aprendizaje automático sirven para un propósito similar. Son una forma de representar palabras como listas de números, donde cada número es como una de esas características o dimensiones para las frutas. Por ejemplo, la palabra "rey" podría representarse por una lista de números como [0.2, -0.4, 0.5, ..., 0.1], donde cada número captura algún aspecto de lo que significa la palabra "rey", como su aspecto de realeza, su género, etc.

Estas listas de números suelen ser bastante largas para capturar los matices en el significado de una palabra; podrían tener cientos de dimensiones. Cuando las palabras tienen significados similares o se usan en contextos similares, sus listas de números se parecerán entre sí cuando realices operaciones matemáticas con ellas. Entonces, por ejemplo, la lista de números para "reina" podría parecerse mucho a "rey", pero ser muy diferente de "manzana".

#### Mejoras varias posibles

Estos números se estiman como consecuencia usualmente secundaria de entrenar un modelo que aprende a predecir la palabra que corresponde luego de una secuencia, o la palabra correspondiente a una bolsa de palabras ignorando la secuencia. Es el caso de los embeddings que usé, por eso esta suma/resta de significados de palabras de una "definición" para ver qué palabra se parece en realidad no tendría por qué funcionar tan bien ya que no era el objetivo real del entrenamiento de ese modelo. Hay varias mejoras posibles en el modelo y en el proceso:

* Usar las definiciones de palabras: Debería poder usar los embeddings para adaptar la tarea definición -> palabra utilizando datos reales de un diccionario. De hecho este fue uno de los principales problemas porque la base de embeddings era bastante gigante, porque contenía una cantidad no menor de palabras relacionadas a lugares/nombres/otros que no me sirven de nada para esta herramienta. Quería filtrarla por un listado de palabras de diccionario, pero info abierta de este estilo hay poca. Terminé usando [Wikcionario](https://es.wiktionary.org/wiki/Wikcionario:Portada) que es la única que encontré realmente opensource, pero fué un webeo y además no está tan completa y tiene harto relleno. 
* Múltiples significados de palabras: El contexto al hacerlo de esta forma se pierde y por lo tanto encontrar alguna manera de tener vectores múltiples puede ser una forma de mejorar el "modelo".
* Usar algún LLM: Ahora liberaron llama2 finetuneada en español (hablando chino) por lo que seguramente se podría utilizar para adaptarlo a este problema. Igual no me corre en el pc, aunque justo desde hoy se encuentra disponible en huggingface una versión cuantizada de llama2 en español [acá](https://huggingface.co/clibrain)

#### Falló la app?

* Por lo que decía del filtro de la base de embeddings utilizando un diccionario de dudosa procedencia, hay palabras que no existen en el modelo y por lo tanto puede tirar error
* También si incluyeron cualquier cosa que no sea palabras separadas por UN espacio, podría fallar. Por ser una demo no le puse mucho cariño a la sanitización del texto que se come la aplicación
